# Day 15: A2UI - Agent 生成动态界面

## 从一个糟糕的体验说起

你开发了一个餐厅推荐 Agent。用户问："帮我找一家附近的西餐厅"

Agent 回复：

```
我找到了以下餐厅：
1. 意大利花园 - 人均 ¥180 - 评分 4.8 - 距离 500m
2. 牛排工坊 - 人均 ¥280 - 评分 4.5 - 距离 800m  
3. 巴黎小馆 - 人均 ¥150 - 评分 4.9 - 距离 1.2km

请告诉我您想选择哪家，我可以帮您预订...
```

用户需要从文字中"脑补"出列表，手动输入选择。这就是纯文字聊天界面的局限。

## 如果 Agent 能生成 UI 呢？

```
┌─────────────────────────────────────────────────┐
│  🍽️ 附近的西餐厅                                │
├─────────────────────────────────────────────────┤
│  🍕 意大利花园  ¥180  ★4.8  500m  [查看] [预订] │
│  🥩 牛排工坊   ¥280  ★4.5  800m  [查看] [预订] │
│  🍝 巴黎小馆   ¥150  ★4.9  1.2km [查看] [预订] │
├─────────────────────────────────────────────────┤
│  筛选: [价格 ▼] [距离 ▼] [评分 ▼]              │
└─────────────────────────────────────────────────┘
```

用户直接点击、筛选、预订。这就是 **A2UI** 要解决的问题。

## 直接生成 HTML 的问题

```python
# ❌ 危险的做法
html = llm.generate("生成一个餐厅列表的 HTML")
render(html)  # 直接执行 LLM 输出的代码
```

问题：
1. **安全风险** - LLM 可能生成恶意 JavaScript
2. **不可控** - 样式混乱，破坏页面布局
3. **不跨平台** - Web/iOS/Android 各需一套

## A2UI 的解决方案

**Agent 不生成代码，只描述"想要什么"。**

```
Agent: "我要一个卡片，包含餐厅名、价格、评分，有一个预订按钮"
客户端: "好，我用我的 Card + Button 组件来渲染"
```

Agent 输出 JSONL 格式的描述，客户端用预定义的安全组件渲染。

---

## 实践 1: 理解 A2UI JSONL 格式

In [ ]:
import json
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field

# A2UI JSONL 包含三种消息类型

# 1. surfaceUpdate - 定义 UI 组件
surface_update = {
    "surfaceUpdate": {
        "surfaceId": "restaurant-finder",
        "components": [
            {
                "id": "title",
                "component": {
                    "Text": {
                        "text": {"literalString": "附近的西餐厅"},
                        "usageHint": "h1"
                    }
                }
            },
            {
                "id": "restaurant-name",
                "component": {
                    "Text": {
                        "text": {"path": "/restaurant/name"},  # 数据绑定
                        "usageHint": "h2"
                    }
                }
            }
        ]
    }
}

# 2. dataModelUpdate - 填充数据
data_update = {
    "dataModelUpdate": {
        "surfaceId": "restaurant-finder",
        "contents": [
            {
                "key": "restaurant",
                "valueMap": [
                    {"key": "name", "valueString": "意大利花园"},
                    {"key": "price", "valueString": "¥180"},
                    {"key": "rating", "valueString": "4.8"}
                ]
            }
        ]
    }
}

# 3. beginRendering - 开始渲染
begin_rendering = {
    "beginRendering": {
        "surfaceId": "restaurant-finder",
        "root": "title"
    }
}

print("A2UI JSONL 消息示例：")
print("=" * 50)
print("\n1️⃣ surfaceUpdate (定义组件):")
print(json.dumps(surface_update, ensure_ascii=False, indent=2))
print("\n2️⃣ dataModelUpdate (填充数据):")
print(json.dumps(data_update, ensure_ascii=False, indent=2))
print("\n3️⃣ beginRendering (开始渲染):")
print(json.dumps(begin_rendering, ensure_ascii=False, indent=2))

---

## 实践 2: 构建 A2UI 生成器

In [ ]:
class A2UIBuilder:
    """A2UI 消息构建器"""
    
    def __init__(self, surface_id: str):
        self.surface_id = surface_id
        self.components = []
        self.data = {}
    
    # 组件方法
    def text(self, id: str, content: str, hint: str = "body") -> 'A2UIBuilder':
        self.components.append({
            "id": id,
            "component": {
                "Text": {
                    "text": {"literalString": content},
                    "usageHint": hint
                }
            }
        })
        return self
    
    def text_bound(self, id: str, data_path: str, hint: str = "body") -> 'A2UIBuilder':
        """数据绑定的文本"""
        self.components.append({
            "id": id,
            "component": {
                "Text": {
                    "text": {"path": data_path},
                    "usageHint": hint
                }
            }
        })
        return self
    
    def button(self, id: str, label: str, action: str) -> 'A2UIBuilder':
        label_id = f"{id}-label"
        self.text(label_id, label)
        self.components.append({
            "id": id,
            "component": {
                "Button": {
                    "child": label_id,
                    "action": {"name": action}
                }
            }
        })
        return self
    
    def text_field(self, id: str, data_path: str, label: str = "") -> 'A2UIBuilder':
        props = {"value": {"path": data_path}}
        if label:
            props["labelText"] = {"literalString": label}
        self.components.append({
            "id": id,
            "component": {"TextField": props}
        })
        return self
    
    def column(self, id: str, children: List[str]) -> 'A2UIBuilder':
        self.components.append({
            "id": id,
            "component": {"Column": {"children": children}}
        })
        return self
    
    def row(self, id: str, children: List[str]) -> 'A2UIBuilder':
        self.components.append({
            "id": id,
            "component": {"Row": {"children": children}}
        })
        return self
    
    def card(self, id: str, child: str) -> 'A2UIBuilder':
        self.components.append({
            "id": id,
            "component": {"Card": {"child": child}}
        })
        return self
    
    def image(self, id: str, url: str) -> 'A2UIBuilder':
        self.components.append({
            "id": id,
            "component": {"Image": {"source": {"literalString": url}}}
        })
        return self
    
    # 数据方法
    def set_data(self, key: str, value: Any) -> 'A2UIBuilder':
        self.data[key] = value
        return self
    
    # 生成 JSONL
    def build(self, root_id: str) -> str:
        lines = []
        
        # surfaceUpdate
        lines.append(json.dumps({
            "surfaceUpdate": {
                "surfaceId": self.surface_id,
                "components": self.components
            }
        }, ensure_ascii=False))
        
        # dataModelUpdate (if data exists)
        if self.data:
            contents = []
            for key, value in self.data.items():
                if isinstance(value, dict):
                    contents.append({
                        "key": key,
                        "valueMap": [{"key": k, "valueString": str(v)} for k, v in value.items()]
                    })
                else:
                    contents.append({"key": key, "valueString": str(value)})
            
            lines.append(json.dumps({
                "dataModelUpdate": {
                    "surfaceId": self.surface_id,
                    "contents": contents
                }
            }, ensure_ascii=False))
        
        # beginRendering
        lines.append(json.dumps({
            "beginRendering": {
                "surfaceId": self.surface_id,
                "root": root_id
            }
        }, ensure_ascii=False))
        
        return "\n".join(lines)


print("✅ A2UIBuilder 类已定义")

---

## 实践 3: 构建餐厅推荐界面

In [ ]:
def create_restaurant_ui(restaurants: List[Dict]) -> str:
    """
    创建餐厅推荐界面
    
    这是 A2UI 的核心用法：根据数据动态生成界面描述
    """
    builder = A2UIBuilder("restaurant-finder")
    
    # 标题
    builder.text("title", "🍽️ 附近的西餐厅", "h1")
    
    # 为每个餐厅创建卡片
    restaurant_cards = []
    for i, r in enumerate(restaurants):
        prefix = f"r{i}"
        
        # 餐厅信息
        builder.text(f"{prefix}-name", f"{r['name']}", "h2")
        builder.text(f"{prefix}-info", f"¥{r['price']} · ★{r['rating']} · {r['distance']}")
        
        # 按钮
        builder.button(f"{prefix}-view", "查看", f"view_restaurant_{i}")
        builder.button(f"{prefix}-book", "预订", f"book_restaurant_{i}")
        builder.row(f"{prefix}-buttons", [f"{prefix}-view", f"{prefix}-book"])
        
        # 卡片内容
        builder.column(f"{prefix}-content", [f"{prefix}-name", f"{prefix}-info", f"{prefix}-buttons"])
        builder.card(f"{prefix}-card", f"{prefix}-content")
        
        restaurant_cards.append(f"{prefix}-card")
    
    # 主容器
    builder.column("restaurant-list", restaurant_cards)
    builder.column("main", ["title", "restaurant-list"])
    
    return builder.build("main")


# 测试数据
restaurants = [
    {"name": "🍕 意大利花园", "price": 180, "rating": 4.8, "distance": "500m"},
    {"name": "🥩 牛排工坊", "price": 280, "rating": 4.5, "distance": "800m"},
    {"name": "🍝 巴黎小馆", "price": 150, "rating": 4.9, "distance": "1.2km"},
]

a2ui_output = create_restaurant_ui(restaurants)

print("生成的 A2UI JSONL:")
print("=" * 60)
for line in a2ui_output.split("\n"):
    msg = json.loads(line)
    msg_type = list(msg.keys())[0]
    if msg_type == "surfaceUpdate":
        comp_count = len(msg["surfaceUpdate"]["components"])
        print(f"📦 surfaceUpdate: {comp_count} 个组件")
    else:
        print(f"📝 {msg_type}")

---

## 实践 4: 与 Gemini 集成

让 LLM 动态生成 A2UI 界面。关键是在 prompt 中包含 A2UI Schema。

In [ ]:
# A2UI Schema 提示词（给 LLM 用）
A2UI_SYSTEM_PROMPT = '''
你是一个 UI 生成助手，使用 A2UI 协议输出界面描述。

## 输出格式
输出 JSONL（每行一个 JSON 对象），按以下顺序：
1. surfaceUpdate - 定义组件
2. dataModelUpdate - 设置数据（可选）
3. beginRendering - 开始渲染

## 可用组件

Text（文本）:
{"id": "xxx", "component": {"Text": {"text": {"literalString": "内容"}, "usageHint": "h1|h2|body"}}}

Button（按钮）:
{"id": "xxx", "component": {"Button": {"child": "文本组件id", "action": {"name": "动作名"}}}}

TextField（输入框）:
{"id": "xxx", "component": {"TextField": {"value": {"path": "/数据/路径"}, "labelText": {"literalString": "标签"}}}}

Column（垂直布局）:
{"id": "xxx", "component": {"Column": {"children": ["子组件id1", "子组件id2"]}}}

Row（水平布局）:
{"id": "xxx", "component": {"Row": {"children": ["子组件id1", "子组件id2"]}}}

Card（卡片）:
{"id": "xxx", "component": {"Card": {"child": "内容组件id"}}}

## 规则
1. 每行一个 JSON 对象
2. 组件 id 必须唯一
3. 引用的子组件必须先定义
4. 最后输出 beginRendering
'''

print("A2UI System Prompt 已定义")
print(f"长度: {len(A2UI_SYSTEM_PROMPT)} 字符")

In [ ]:
import os
import sys
sys.path.append('..')

# 加载环境变量
from dotenv import load_dotenv
load_dotenv('../.env')

try:
    import google.generativeai as genai
    
    api_key = os.getenv('GOOGLE_API_KEY')
    if api_key:
        genai.configure(api_key=api_key)
        print("✅ Gemini API 已配置")
    else:
        print("⚠️ 未找到 GOOGLE_API_KEY，将使用模拟模式")
except ImportError:
    print("⚠️ google-generativeai 未安装，将使用模拟模式")

In [ ]:
async def generate_a2ui_with_gemini(user_request: str) -> str:
    """
    使用 Gemini 生成 A2UI
    
    在实际应用中，这个函数会被 Agent 调用
    """
    try:
        model = genai.GenerativeModel(
            model_name="gemini-2.0-flash-exp",
            system_instruction=A2UI_SYSTEM_PROMPT
        )
        
        response = await model.generate_content_async(
            f"用户请求: {user_request}\n\n请生成对应的 A2UI 界面。"
        )
        
        return response.text
    except Exception as e:
        print(f"Gemini API 错误: {e}")
        # 返回模拟数据
        return simulate_a2ui_response(user_request)


def simulate_a2ui_response(user_request: str) -> str:
    """模拟 LLM 响应（用于无 API 时演示）"""
    if "登录" in user_request:
        return A2UIBuilder("login") \
            .text("title", "用户登录", "h1") \
            .text_field("username", "/user/name", "用户名") \
            .text_field("password", "/user/password", "密码") \
            .button("submit", "登录", "submit_login") \
            .column("form", ["title", "username", "password", "submit"]) \
            .card("card", "form") \
            .build("card")
    elif "天气" in user_request:
        return A2UIBuilder("weather") \
            .text("title", "🌤️ 今日天气", "h1") \
            .text("city", "北京", "h2") \
            .text("temp", "25°C 晴") \
            .text("humidity", "湿度 45%") \
            .button("refresh", "刷新", "refresh_weather") \
            .column("info", ["title", "city", "temp", "humidity", "refresh"]) \
            .card("card", "info") \
            .build("card")
    else:
        return A2UIBuilder("default") \
            .text("msg", f"收到: {user_request}") \
            .build("msg")


# 测试
import asyncio

async def test_generation():
    requests = ["生成一个登录表单", "显示今天的天气"]
    for req in requests:
        print(f"\n📨 请求: {req}")
        print("-" * 40)
        result = await generate_a2ui_with_gemini(req)
        # 显示摘要
        lines = result.strip().split("\n")
        print(f"✅ 生成了 {len(lines)} 行 JSONL")

await test_generation()

---

## 实践 5: 使用 A2UI Composer

Google 提供了在线编辑器 [A2UI Composer](https://a2ui-editor.ag-ui.com/)，可以可视化创建 A2UI 组件。

### 使用方法

1. 访问 https://a2ui-editor.ag-ui.com/
2. 描述你想要的 UI（如："一个用户注册表单，包含用户名、邮箱、密码"）
3. 点击 Create 生成
4. 复制生成的 JSON 到你的 Agent prompt 中

### 生成的 JSON 可以直接用于 Agent

```python
# 从 A2UI Composer 复制的组件定义
REGISTRATION_FORM_TEMPLATE = '''{
    "surfaceUpdate": {
        "surfaceId": "registration",
        "components": [...]
    }
}'''

# Agent 可以直接使用这个模板，或者让 LLM 基于此生成变体
```

In [ ]:
# 模拟从 A2UI Composer 获取的模板

REGISTRATION_TEMPLATE = {
    "components": [
        {"id": "title", "component": {"Text": {"text": {"literalString": "创建账号"}, "usageHint": "h1"}}},
        {"id": "username", "component": {"TextField": {"value": {"path": "/user/username"}, "labelText": {"literalString": "用户名"}}}},
        {"id": "email", "component": {"TextField": {"value": {"path": "/user/email"}, "labelText": {"literalString": "邮箱"}}}},
        {"id": "password", "component": {"TextField": {"value": {"path": "/user/password"}, "labelText": {"literalString": "密码"}}}},
        {"id": "confirm", "component": {"TextField": {"value": {"path": "/user/confirmPassword"}, "labelText": {"literalString": "确认密码"}}}},
        {"id": "btn-text", "component": {"Text": {"text": {"literalString": "注册"}}}},
        {"id": "submit", "component": {"Button": {"child": "btn-text", "action": {"name": "register"}}}},
        {"id": "form", "component": {"Column": {"children": ["title", "username", "email", "password", "confirm", "submit"]}}},
        {"id": "card", "component": {"Card": {"child": "form"}}}
    ]
}

def use_template(template: dict, surface_id: str, root_id: str) -> str:
    """使用预定义模板生成 A2UI"""
    lines = [
        json.dumps({"surfaceUpdate": {"surfaceId": surface_id, "components": template["components"]}}, ensure_ascii=False),
        json.dumps({"beginRendering": {"surfaceId": surface_id, "root": root_id}}, ensure_ascii=False)
    ]
    return "\n".join(lines)


print("使用 A2UI Composer 模板:")
print("=" * 50)
result = use_template(REGISTRATION_TEMPLATE, "registration", "card")
print(f"生成了 {len(result.split(chr(10)))} 行 JSONL")
print(f"包含 {len(REGISTRATION_TEMPLATE['components'])} 个组件")

---

## 实践 6: ASCII 渲染器（可视化验证）

在真实应用中，A2UI 由客户端渲染器（如 React、Flutter）渲染。这里用 ASCII Art 模拟，帮助验证生成的 A2UI 是否正确。

In [ ]:
class ASCIIRenderer:
    """将 A2UI JSONL 渲染为 ASCII Art"""
    
    def __init__(self):
        self.components = {}
        self.root_id = None
    
    def parse(self, jsonl: str):
        for line in jsonl.strip().split("\n"):
            msg = json.loads(line)
            if "surfaceUpdate" in msg:
                for comp in msg["surfaceUpdate"]["components"]:
                    self.components[comp["id"]] = comp["component"]
            elif "beginRendering" in msg:
                self.root_id = msg["beginRendering"]["root"]
    
    def render_component(self, comp_id: str, indent: int = 0) -> List[str]:
        if comp_id not in self.components:
            return []
        
        comp = self.components[comp_id]
        prefix = "  " * indent
        lines = []
        
        if "Text" in comp:
            text = comp["Text"]["text"].get("literalString", "[数据绑定]")
            hint = comp["Text"].get("usageHint", "body")
            if hint == "h1":
                lines.append(f"{prefix}╔{'═' * (len(text) + 2)}╗")
                lines.append(f"{prefix}║ {text} ║")
                lines.append(f"{prefix}╚{'═' * (len(text) + 2)}╝")
            elif hint == "h2":
                lines.append(f"{prefix}▸ {text}")
            else:
                lines.append(f"{prefix}  {text}")
        
        elif "Button" in comp:
            child = comp["Button"]["child"]
            if child in self.components and "Text" in self.components[child]:
                label = self.components[child]["Text"]["text"].get("literalString", "Button")
            else:
                label = "Button"
            lines.append(f"{prefix}[ {label} ]")
        
        elif "TextField" in comp:
            label = comp["TextField"].get("labelText", {}).get("literalString", "")
            if label:
                lines.append(f"{prefix}{label}: [__________]")
            else:
                lines.append(f"{prefix}[__________]")
        
        elif "Column" in comp:
            for child_id in comp["Column"].get("children", []):
                lines.extend(self.render_component(child_id, indent))
        
        elif "Row" in comp:
            row_items = []
            for child_id in comp["Row"].get("children", []):
                child_lines = self.render_component(child_id, 0)
                if child_lines:
                    row_items.append(child_lines[0].strip())
            lines.append(f"{prefix}{' | '.join(row_items)}")
        
        elif "Card" in comp:
            child_id = comp["Card"]["child"]
            inner = self.render_component(child_id, indent + 1)
            if inner:
                max_len = max(len(l) for l in inner)
                lines.append(f"{prefix}┌{'─' * (max_len - indent * 2 + 2)}┐")
                for l in inner:
                    lines.append(f"{prefix}│{l[indent*2:]}│")
                lines.append(f"{prefix}└{'─' * (max_len - indent * 2 + 2)}┘")
        
        return lines
    
    def render(self, jsonl: str) -> str:
        self.components = {}
        self.parse(jsonl)
        if self.root_id:
            return "\n".join(self.render_component(self.root_id))
        return ""


# 渲染餐厅列表
renderer = ASCIIRenderer()
print("🖥️ ASCII 渲染结果:")
print()
print(renderer.render(a2ui_output))

---

## 与 Day 14 A2A 的整合

A2UI 可以通过 A2A 协议传输：

```
┌──────────────┐      A2A       ┌──────────────┐
│  Host Agent  │ ◄────────────► │   UI Agent   │
│  (客户端)     │    JSONL       │  (生成 A2UI) │
└──────────────┘                └──────────────┘
       │
       ▼
  渲染 A2UI
  (React/Flutter)
```

Host Agent 可以调用专门的 UI Agent 获取界面描述，然后渲染给用户。

In [ ]:
# 模拟 A2A 消息中携带 A2UI

def wrap_a2ui_in_a2a(a2ui_jsonl: str, task_id: str = "task-001") -> dict:
    """
    将 A2UI 封装为 A2A 响应
    
    A2A 协议支持 artifact 类型的消息部分，A2UI 可以作为 artifact 传输
    """
    return {
        "jsonrpc": "2.0",
        "id": task_id,
        "result": {
            "id": task_id,
            "status": {"state": "completed"},
            "artifacts": [
                {
                    "name": "ui",
                    "parts": [
                        {
                            "type": "a2ui",
                            "mimeType": "application/x-a2ui+jsonl",
                            "content": a2ui_jsonl
                        }
                    ]
                }
            ]
        }
    }


# 示例
a2a_response = wrap_a2ui_in_a2a(a2ui_output)
print("A2A 响应（包含 A2UI）:")
print(json.dumps(a2a_response, ensure_ascii=False, indent=2)[:500] + "...")

---

## 总结

今天学习了 A2UI 的核心概念和用法：

| 概念 | 说明 |
|------|------|
| **声明式** | Agent 描述"要什么"，不生成可执行代码 |
| **JSONL** | 每行一个 JSON，支持流式传输 |
| **三种消息** | surfaceUpdate → dataModelUpdate → beginRendering |
| **组件系统** | Text, Button, TextField, Column, Row, Card 等 |
| **ID 引用** | 组件通过 ID 相互引用，形成树结构 |

### 资源

- [A2UI 官网](https://a2ui.org/)
- [A2UI GitHub](https://github.com/google/A2UI)
- [A2UI Composer](https://a2ui-editor.ag-ui.com/) - 可视化编辑器
- [CopilotKit A2UI 教程](https://www.copilotkit.ai/blog/how-to-build-agent-to-user-interface-a2ui-agents-using-a2a-ag-ui)